# Tensorflow 2.0

저희는 텐서플로 2.0 버전을 기준으로 합니다. 이번 튜토리얼에서는 간략한 예제로 텐서플로의 큰 그림을 잡고자 합니다.

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras import models, layers, losses, metrics, optimizers

In [2]:
tf.__version__

'2.0.0'

텐서플로는 두 가지 방식의 페러다임(?)을 지원합니다.
1. 그래프를 먼저 구축 및 빌드(컴파일)하고 그 그래프에 입력을 주는 방식.
2. 대략적인 형태만 만들고, 입력이 들어가게 되면 그래프가 구축되는 방식(컴파일 없음).

저희는 2번을 볼 텐데요, tensorflow 1.x에서는 1번이 기본이었지만, 2.0에서는 2번이 기본이 되었습니다. 텐서플로는 이것을 eager execution이라고 부르는데요,
그래프를 그리고 session.run() 해야만 결과를 얻던 기존의 방식과 다르게 바로바로 결과를 얻을 수 있는 방식을 의미합니다. 이것이 활성화되있는지 알고 싶다면 다음을 호출합니다.

In [18]:
# 2.0에서는 default값이 true입니다.
tf.executing_eagerly()

True

## 연습용 MNIST 데이터

먼저 연습용으로 쓸 MNIST 데이터를 로드합니다.

In [3]:
from tensorflow.keras.datasets.mnist import load_data

In [4]:
trainset, testset = load_data()

In [5]:
x_train, y_train = trainset
x_test, y_test = testset

In [6]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


다음은 간단히 데이터를 standardize하는 과정입니다.

In [7]:
# mean = 0, std = 0.5
x_train = (x_train.astype(np.float32) - 128) / 256
x_test = (x_test.astype(np.float32) - 128) / 256

## 모델 구성

먼저 빠르게 모델을 만들어 보겠습니다.

In [8]:
# 먼저, 이미지를 벡터로 펼치고,
# 128개의 hidden unit을 가지는 dense(fully connected) layer
# 128개의 hidden unit을 가지는 dense(fully connected) layer
# 10개의 hidden unit을 가지는 dense(fully connected) layer

myfirstmodel = models.Sequential([
    layers.Flatten(),
    layers.Dense(128, activation=tf.nn.tanh),
    layers.Dense(128, activation=tf.nn.tanh),
    layers.Dense(10, activation=tf.nn.softmax),
])

지금, 모델의 레이어를 구성했는데요, 모델을 구성했으면, 이제 모델을 빌드해야 합니다.

빌드하는 방법은 두 가지가 있는데요.
1. .build() 호출.
2. .fit() 또는 .call() 호출.

Keras에는 모델을 구성했다고 해서 모델이 완성되는 것이 아닙니다. .build()를 호출하거나, forward propagation을 한번 이상 호출하게 되면 모델이 비로소 빌드가 됩니다.

In [9]:
myfirstmodel.build(input_shape=(None, 28, 28))

Keras에는 빌드된 그래프를 간략히 요약해서 볼 수 있는 summary() 메소드가 존재합니다. 이 메소드는 빌드된 모델에 대해서만 호출이 가능합니다.

In [10]:
myfirstmodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  16512     
_________________________________________________________________
dense_2 (Dense)              multiple                  1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


다음으로, model을 학습시킬때 이용할 optimizer와 criterion (loss 함수), metric을 정의합니다.

여기서, categorical cross entropy는 두 가지가 있습니다.
- CategoricalCrossentropy
- SparseCategoricalCrossentropy

Sparse가 붙은 애들은 y_true값으로 one-hot된 라벨을 주지 않고, 그냥 라벨 인코딩 상태로만 넘겨줘야 하며, Sparse가 아닌 애들을 one-hot된 y_true값을 넘겨줘야 합니다.
y_preds는 둘 다 형태가 같습니다.

Sparse는 accuracy에도 해당됩니다. (SpareseCategoricalAccuracy, CategoricalAccuracy)

In [11]:
optimizer = optimizers.Adam(learning_rate=1e-3)
criterion = losses.SparseCategoricalCrossentropy()
metric = metrics.SparseCategoricalAccuracy()

이제 모델을 주어진 optimizer, criterion, metric으로 컴파일해 봅시다. 컴파일을 해야 fit()을 호출해서 모델을 학습시킬 수 있죠.

In [12]:
myfirstmodel.compile(optimizer, loss=criterion, metrics=[metric])

이제 준비가 끝났습니다. 이제 fit()을 호출해서 모델을 학습시킬 일만 남았습니다.

저는 개인적으로 fit()을 호출해서 모델을 학습시키는 방법을 좋아하지 않습니다. 따라서, fit()을 호출해서 모델을 학습시키는 것은 이번 튜토리얼이 아마 마지막이 될 수도 있습니다.
지금은 개략적인 tensorflow 2.0을 보여주는 것이기에 fit()을 사용하겠습니다.

In [13]:
myfirstmodel.fit(x_train, y_train, validation_split=0.2, epochs=10, batch_size=128)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 1s 29us/sample - loss: 0.4081 - sparse_categorical_accuracy: 0.8813 - val_loss: 0.2329 - val_sparse_categorical_accuracy: 0.9329
Epoch 2/10
48000/48000 [==============================] - 1s 14us/sample - loss: 0.2017 - sparse_categorical_accuracy: 0.9403 - val_loss: 0.1761 - val_sparse_categorical_accuracy: 0.9498
Epoch 3/10
48000/48000 [==============================] - 1s 13us/sample - loss: 0.1450 - sparse_categorical_accuracy: 0.9560 - val_loss: 0.1324 - val_sparse_categorical_accuracy: 0.9618
Epoch 4/10
48000/48000 [==============================] - 1s 13us/sample - loss: 0.1118 - sparse_categorical_accuracy: 0.9661 - val_loss: 0.1219 - val_sparse_categorical_accuracy: 0.9650
Epoch 5/10
48000/48000 [==============================] - 1s 14us/sample - loss: 0.0938 - sparse_categorical_accuracy: 0.9719 - val_loss: 0.1088 - val_sparse_categorical_accuracy: 0.9688
Epoch 6/10
4800

모델을 학습시켰으면, 테스트를 해봐야 겠죠? 그 메소드는 evaluate()입니다.

In [17]:
myfirstmodel.evaluate(x_test, y_test, batch_size=128, verbose=0)
# loss, accuracy

[0.09244314468353987, 0.9711]